# Writing LDC solution files for galactic binaries

Maude, 2022/09/30, adapted from Michele's first version (2018/01/24)

In [1]:
import yaml
import numpy
import csv
import pandas

In [2]:
entry = {'author': 'Maude Le Jeune',
         'e-mail': 'lejeune@lejeune.org',
         'date': '2022/09/30',           # standardize?
         'challenge': 'LDC2a',
         'dataset': 'LDC2a-training-GB'}

## A point estimate, from a Python dict

We use standard LVC units, except for frequency, which is given in mHz to avoid potential precision issues

In [3]:
gb = {  'Amplitude' :4.065235140346656e-23, 
        'Frequency' :2.1869889108004115, 
        'FrequencyDerivative':4.837869734659327e-17, 
        'EclipticLatitude': -0.3982801352498764, 
        'EclipticLongitude': 4.4946831814774795, 
        'Inclination': 1.0208838997376564, 
        'Polarization': 3.737792079072995, 
        'InitialPhase': 5.6307387805247435, 
     }
units = {  
        'Amplitude' :1, 
        'Frequency' :'mHz',
        'FrequencyDerivative':'Hz',
        'EclipticLatitude': 'rad',
        'EclipticLongitude': 'rad',
        'Inclination': 'rad',
        'Polarization': 'rad', 
        'InitialPhase': 'rad' 
     }

Use multiple list items for multiple sources.

In [4]:
entry['estimates'] = [gb]
entry['units'] = units

In [5]:
open('lejeune-ldc2a-training-gb.yaml','w').write(yaml.dump(entry, default_flow_style=False))

591

In [6]:
!cat lejeune-ldc2a-training-gb.yaml

author: Maude Le Jeune
challenge: LDC2a
dataset: LDC2a-training-GB
date: 2022/09/30
e-mail: lejeune@lejeune.org
estimates:
- Amplitude: 4.065235140346656e-23
  EclipticLatitude: -0.3982801352498764
  EclipticLongitude: 4.4946831814774795
  Frequency: 2.1869889108004115
  FrequencyDerivative: 4.837869734659327e-17
  Inclination: 1.0208838997376564
  InitialPhase: 5.6307387805247435
  Polarization: 3.737792079072995
units:
  Amplitude: 1
  EclipticLatitude: rad
  EclipticLongitude: rad
  Frequency: mHz
  FrequencyDerivative: Hz
  Inclination: rad
  InitialPhase: rad
  Polarization: rad


### With errors (here we make them up, but you would get them from elsewhere)

In [7]:
gb_werr = {key: '{} +/- {}'.format(value, value * 0.01) if isinstance(value, float) else value
             for key, value in gb.items()}

In [8]:
entry['estimates'] = [gb_werr]
entry['notes'] = '1-sigma errors'

In [9]:
open('lejeune-ldc2a-training-gb_werr.yaml','w').write(yaml.dump(entry, default_flow_style=False))

814

In [10]:
!cat lejeune-ldc2a-training-gb_werr.yaml

author: Maude Le Jeune
challenge: LDC2a
dataset: LDC2a-training-GB
date: 2022/09/30
e-mail: lejeune@lejeune.org
estimates:
- Amplitude: 4.065235140346656e-23 +/- 4.065235140346656e-25
  EclipticLatitude: -0.3982801352498764 +/- -0.003982801352498764
  EclipticLongitude: 4.4946831814774795 +/- 0.044946831814774796
  Frequency: 2.1869889108004115 +/- 0.021869889108004116
  FrequencyDerivative: 4.837869734659327e-17 +/- 4.837869734659327e-19
  Inclination: 1.0208838997376564 +/- 0.010208838997376565
  InitialPhase: 5.6307387805247435 +/- 0.05630738780524744
  Polarization: 3.737792079072995 +/- 0.03737792079072995
notes: 1-sigma errors
units:
  Amplitude: 1
  EclipticLatitude: rad
  EclipticLongitude: rad
  Frequency: mHz
  FrequencyDerivative: Hz
  Inclination: rad
  InitialPhase: rad
  Polarization: rad


### With ranges

In [11]:
gb_range = {key: '{} + {} - {}'.format(value, value * 0.02, value * 0.01) if isinstance(value, float) else value
             for key, value in gb.items()}

In [12]:
entry['estimates'] = [gb_range]
entry['notes'] = '90% symmetric credible interval'

In [13]:
open('lejeune-ldc2a-sangria-gb_range.yaml','w').write(yaml.dump(entry, default_flow_style=False))

996

In [14]:
!cat lejeune-ldc2a-sangria-gb_range.yaml

author: Maude Le Jeune
challenge: LDC2a
dataset: LDC2a-training-GB
date: 2022/09/30
e-mail: lejeune@lejeune.org
estimates:
- Amplitude: 4.065235140346656e-23 + 8.130470280693312e-25 - 4.065235140346656e-25
  EclipticLatitude: -0.3982801352498764 + -0.007965602704997528 - -0.003982801352498764
  EclipticLongitude: 4.4946831814774795 + 0.08989366362954959 - 0.044946831814774796
  Frequency: 2.1869889108004115 + 0.04373977821600823 - 0.021869889108004116
  FrequencyDerivative: 4.837869734659327e-17 + 9.675739469318655e-19 - 4.837869734659327e-19
  Inclination: 1.0208838997376564 + 0.02041767799475313 - 0.010208838997376565
  InitialPhase: 5.6307387805247435 + 0.11261477561049488 - 0.05630738780524744
  Polarization: 3.737792079072995 + 0.0747558415814599 - 0.03737792079072995
notes: 90% symmetric credible interval
units:
  Amplitude: 1
  EclipticLatitude: rad
  EclipticLongitude: rad
  Frequency: mHz
  FrequencyDerivative: Hz
  Inclination: rad
  InitialPhase: rad
  Polarization: rad


## A posterior table, from a numpy array

We assume that: 
- overlapping sources have been sampled together such that a chain contains 8 x n_source columns. 
- each chain correspond to a given frequency range [fmin-fmax], which potentially overlaps with others, such that a source can appear twice in the ensemble chain files. 

Thus, a posterior table has 8 x n_source columns named after parameter name with a suffix corresponding to a given source index. The source index can be arbitrarily chosen. 

In [15]:
data = numpy.array([
       [-2.24871916e+01,  4.00034633, -5.29683032e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  6.02877872e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24871916e+01,  4.00034633, -5.29683032e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  6.02877872e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24871916e+01,  4.00034633, -5.29683032e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  6.02877872e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24331589e+01,  4.00035668, -1.39371732e-15,1.51786749e-01,  4.84519246e+00, -6.22606458e-01,1.19964789e+01,  3.97795495e-01, -2.23873624e+01,4.00186603,  8.38575976e-16, -1.28895008e-01,4.73397643e+00,  5.04903712e-01,  7.70038613e+00,1.27287409e+00],
       [-2.24871916e+01,  4.00034633, -5.35511713e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  6.03759765e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24331589e+01,  4.00035668, -1.39371732e-15,1.51786749e-01,  4.84519246e+00, -6.22606458e-01,1.19964789e+01,  3.97795495e-01, -2.23873624e+01,4.00186603,  8.38575976e-16, -1.28895008e-01,4.73397643e+00,  5.04903712e-01,  7.70038613e+00,1.27287409e+00],
       [-2.24871916e+01,  4.00034633, -5.33625563e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  5.98482650e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24871916e+01,  4.00034633, -5.33625563e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  5.98482650e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01],
       [-2.24331589e+01,  4.00035668, -1.39371732e-15,1.51786749e-01,  4.84519246e+00, -6.22606458e-01,1.19964789e+01,  3.97795495e-01, -2.23873624e+01,4.00186603,  8.38575976e-16, -1.49169611e-01,4.73397643e+00,  5.04903712e-01,  7.70038613e+00,1.27287409e+00],
       [-2.24871916e+01,  4.00034633, -5.65145097e-16,2.69702436e-01,  4.79450637e+00, -7.64330189e-01,1.19741715e+01, -2.57331129e+00, -2.24712398e+01,4.00186568,  5.87696213e-16,  8.66623674e-02,4.69696659e+00,  7.29309345e-01,  1.11261889e+01,9.57692377e-01]])


In [16]:
base_names = ['Amplitude', 'Frequency', 'FrequencyDerivative', 'EclipticLatitude', 'EclipticLongitude',
             'Inclination', 'Polarization', 'InitialPhase']
names = [n+'_'+str(index) for index in [5482, 4765] for n in base_names]
recarray = numpy.rec.fromarrays(list(data.T), names=names)

In [17]:
numpy.savetxt('gb-4.000-4.002-pdf.csv', recarray,
              header=','.join(recarray.dtype.names),
              delimiter=',', fmt='%.8e')

In [18]:
!cat gb-4.000-4.002-pdf.csv

# Amplitude_5482,Frequency_5482,FrequencyDerivative_5482,EclipticLatitude_5482,EclipticLongitude_5482,Inclination_5482,Polarization_5482,InitialPhase_5482,Amplitude_4765,Frequency_4765,FrequencyDerivative_4765,EclipticLatitude_4765,EclipticLongitude_4765,Inclination_4765,Polarization_4765,InitialPhase_4765
-2.24871916e+01,4.00034633e+00,-5.29683032e-16,2.69702436e-01,4.79450637e+00,-7.64330189e-01,1.19741715e+01,-2.57331129e+00,-2.24712398e+01,4.00186568e+00,6.02877872e-16,8.66623674e-02,4.69696659e+00,7.29309345e-01,1.11261889e+01,9.57692377e-01
-2.24871916e+01,4.00034633e+00,-5.29683032e-16,2.69702436e-01,4.79450637e+00,-7.64330189e-01,1.19741715e+01,-2.57331129e+00,-2.24712398e+01,4.00186568e+00,6.02877872e-16,8.66623674e-02,4.69696659e+00,7.29309345e-01,1.11261889e+01,9.57692377e-01
-2.24871916e+01,4.00034633e+00,-5.29683032e-16,2.69702436e-01,4.79450637e+00,-7.64330189e-01,1.19741715e+01,-2.57331129e+00,-2.24712398e+01,4.00186568e+00,6.02877872e-16,8.66623674e-02,4.69696659e+00,7.

Use multiple table files for multiple frequency ranges

In [19]:
entry = {'author': 'Maude Le Jeune',
         'e-mail': 'lejeune@lejeune.org',
         'date': '2018/03/20',           # standardize?
         'challenge': 'LDC2a',
         'dataset': 'LDC2a-training-GB',
         'posteriors': ['gb-4.000-4.002-pdf.csv']}

In [20]:
open('lejeune-ldc2a-training-gb_posteriors.yaml','w').write(yaml.dump(entry, default_flow_style=False))

149

In [21]:
!cat lejeune-ldc2a-training-gb_posteriors.yaml

author: Maude Le Jeune
challenge: LDC2a
dataset: LDC2a-training-GB
date: 2018/03/20
e-mail: lejeune@lejeune.org
posteriors:
- gb-4.000-4.002-pdf.csv


## A posterior table, from a Pandas DataFrame

In [22]:
df = pandas.DataFrame(data,columns=names)

In [23]:
df

,Amplitude_5482,Frequency_5482,FrequencyDerivative_5482,EclipticLatitude_5482,EclipticLongitude_5482,Inclination_5482,Polarization_5482,InitialPhase_5482,Amplitude_4765,Frequency_4765,FrequencyDerivative_4765,EclipticLatitude_4765,EclipticLongitude_4765,Inclination_4765,Polarization_4765,InitialPhase_4765
0,-22.487192,4.000346,-5.296830e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,6.028779e-16,0.086662,4.696967,0.729309,11.126189,0.957692
1,-22.487192,4.000346,-5.296830e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,6.028779e-16,0.086662,4.696967,0.729309,11.126189,0.957692
2,-22.487192,4.000346,-5.296830e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,6.028779e-16,0.086662,4.696967,0.729309,11.126189,0.957692
3,-22.433159,4.000357,-1.393717e-15,0.151787,4.845192,-0.622606,11.996479,0.397795,-22.387362,4.001866,8.385760e-16,-0.128895,4.733976,0.504904,7.700386,1.272874
4,-22.487192,4.000346,-5.355117e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,6.037598e-16,0.086662,4.696967,0.729309,11.126189,0.957692
5,-22.433159,4.000357,-1.393717e-15,0.151787,4.845192,-0.622606,11.996479,0.397795,-22.387362,4.001866,8.385760e-16,-0.128895,4.733976,0.504904,7.700386,1.272874
6,-22.487192,4.000346,-5.336256e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,5.984827e-16,0.086662,4.696967,0.729309,11.126189,0.957692
7,-22.487192,4.000346,-5.336256e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,5.984827e-16,0.086662,4.696967,0.729309,11.126189,0.957692
8,-22.433159,4.000357,-1.393717e-15,0.151787,4.845192,-0.622606,11.996479,0.397795,-22.387362,4.001866,8.385760e-16,-0.149170,4.733976,0.504904,7.700386,1.272874
9,-22.487192,4.000346,-5.651451e-16,0.269702,4.794506,-0.764330,11.974172,-2.573311,-22.471240,4.001866,5.876962e-16,0.086662,4.696967,0.729309,11.126189,0.957692


In [24]:
df.to_csv('gb-4.000-4.002-pdf.csv',index=False)

In [25]:
!cat gb-4.000-4.002-pdf.csv

Amplitude_5482,Frequency_5482,FrequencyDerivative_5482,EclipticLatitude_5482,EclipticLongitude_5482,Inclination_5482,Polarization_5482,InitialPhase_5482,Amplitude_4765,Frequency_4765,FrequencyDerivative_4765,EclipticLatitude_4765,EclipticLongitude_4765,Inclination_4765,Polarization_4765,InitialPhase_4765
-22.4871916,4.00034633,-5.29683032e-16,0.269702436,4.79450637,-0.764330189,11.9741715,-2.57331129,-22.4712398,4.00186568,6.02877872e-16,0.0866623674,4.69696659,0.729309345,11.1261889,0.957692377
-22.4871916,4.00034633,-5.29683032e-16,0.269702436,4.79450637,-0.764330189,11.9741715,-2.57331129,-22.4712398,4.00186568,6.02877872e-16,0.0866623674,4.69696659,0.729309345,11.1261889,0.957692377
-22.4871916,4.00034633,-5.29683032e-16,0.269702436,4.79450637,-0.764330189,11.9741715,-2.57331129,-22.4712398,4.00186568,6.02877872e-16,0.0866623674,4.69696659,0.729309345,11.1261889,0.957692377
-22.4331589,4.00035668,-1.39371732e-15,0.151786749,4.84519246,-0.622606458,11.9964789,0.397795495,-22.3873624